In [428]:
## Josh Blaz -- LOTR
import nltk
import re
import urllib.request
import lxml.html as lh
import io
import requests
import os
import glob
from ggplot import *

#NOTE: Sentiment140 Polarity values: 0: negative, 2: neutral, 4: positive

**NOTE: Elvish text is translated awkwardly into the .txt format    
</br>
IE:   
</br>
►M MPR -F+MTRX MP ft PPtK P&RMPht: P. t. The last Two runes are the initials of Thror and Thrain.**

In [431]:
# Used Chapterize to split books into chapters 
## https://github.com/JonathanReeve/chapterize
### Chapterize didn't work 100% perfectly, so I had to go through and the prologues back in when it cut them out

# These are lists containing strings of every chapter for each book
silm_chapters = []
hobbit_chapters = []
fellowship_chapters = []
twotowers_chapters = []
return_chapters = []

# Paths to directories storing book chapters
list_of_paths = ['/Users/blaz/Desktop/LOTR/silmarillion-chapters', '/Users/blaz/Desktop/LOTR/hobbit-chapters',\
                '/Users/blaz/Desktop/LOTR/fellowship-chapters', '/Users/blaz/Desktop/LOTR/twotowers-chapters',\
                '/Users/blaz/Desktop/LOTR/return-chapters']

for path in list_of_paths: # iterate through the list of folder paths for each book
    for file in sorted(glob.glob(os.path.join(path,'*.txt'))): # This gives us a sorted list of the files in each directory                                                         
        f = open(file, 'r') # open and read file               # allowing us to read in the chapters in order.
        txt = f.read()
    
        ## determine which path we're using and append it to the correct book chapter list
        if path == '/Users/blaz/Desktop/LOTR/silmarillion-chapters': 
            silm_chapters.append(txt)
        elif path == '/Users/blaz/Desktop/LOTR/hobbit-chapters':
            hobbit_chapters.append(txt)
        elif path == '/Users/blaz/Desktop/LOTR/fellowship-chapters': 
            fellowship_chapters.append(txt)
        elif path == '/Users/blaz/Desktop/LOTR/twotowers-chapters': 
            twotowers_chapters.append(txt)
        elif path == '/Users/blaz/Desktop/LOTR/return-chapters': 
            return_chapters.append(txt)

print(len(silm_chapters))
print(silm_chapters[len(silm_chapters)-1])

27

Of the Silmarils and the Unrest of the 
Noldor 

In that time were made those things that afterwards 
were most renowned of all the works of the Elves. For 
Feanor, being come to his full might, was filled with a new 
thought, or it may be that some shadow of foreknowledge 
came to him of the doom that drew near; and he pondered 
how the light of the Trees, the glory of the Blessed Realm, 
might be preserved imperishable. Then he began a long 
and secret labour, and he summoned all his lore, and his 
power, and his subtle skill; and at the end of all he made the 
Silmarils. 

As three great Jewels they were in form. But not until 
the End, when Feanor shall return who perished ere the 
Sun was made, and sits now in the Halls of Awaiting and 
comes no more among his kin; not until the Sun passes 
and the Moon falls, shall it be known of what substance they 
were made. Like the crystal of diamonds it appeared, and 
yet was more strong than adamant, so that no violence 
could mar it o

# Sentiment Analysis

**Now, for each book, we have lists containing all of the chapters for that book.  
</br>
This was accomplished by iterating through the directories that store the chapters for each book using the "glob" and "os" modules.**

In [419]:
"""
Function that segments given chapter into a page-sized (2940 character) segments to be sent to the API.

Params - chapter is a chapter of a book to be broken into segments

Returns a list of (string) segments of the chapter.
"""
def Segmenter(chapter, cut):
    segments = []
    # start and end indices for segmenting the text
    start = 0
    end = cut
    while end < len(chapter) + cut:
        segments.append(chapter[start:end])
        start = end
        end = end + cut
    
    return segments #segments of input chapter

**This function allows us to split the chapters of each book into segments to send in our HTTP-Post JSON requests.   
</br>
I chose 2940 as the length because this is the exact number of characters per page (including spaces) in my copy of Fellowship of the Ring. **

In [420]:
# Lists of Lists of Lists storing all segments of all chapters for each book
# [[chapter1 segment 0-2500, chap1, segmenet 2500-5000]... [chapter2 segment0-2500, ...]...]
silm_segments = []
hobbit_segments = []
fellowship_segments = []
twotowers_segments = []
return_segments = []

# List containing the lists storing each books' chapters
list_of_books = [silm_chapters, hobbit_chapters, fellowship_chapters, twotowers_chapters, return_chapters]
# List allowing us to access the segment lists
list_of_segments = [silm_segments, hobbit_segments, fellowship_segments, twotowers_segments, return_segments]

# Access the lists above so that we can iteratively segment each chapter in the corpus
for i in range(len(list_of_books)):
    for book in list_of_books:
        for chapter in list_of_books[i]:
            list_of_segments[i].append(Segmenter(chapter,2940))
            

**In the cell above I create lists of lists for segments of each chapter of each corpus or book, and append to them using my "Segmenter"
function, storing them neatly like this will allow me to iteratively query the API server. **

In [421]:
"""
Function that sends segments of 1 chapter through the Sentiment140 API.

In order to do so, it creates and appends segments to a JSON file, then posts the JSON queries to the API server
using requests module (using an HTTP Post)

Returns a list of polarities for segments of the chapter, as well as the polarity average for the chapter

Note: Maximum of 700,000 characters per API request, though this shouldn't be a problem
"""

def Polarity(chapter_segments): # segments of a single chapter
    request = {'data':[]}
    polarityList = []
    counter = 0
    for segment in chapter_segments: # Fill JSON
        request['data'].append({'text':segment})
    r = requests.post('http://www.sentiment140.com/api/bulkClassifyJson?appid=blaz_j1@denison.edu', json=request)
    jso = r.json()
    for i in range(len(request['data'])-1):
        polarityList.append(jso['data'][i]['polarity'])
    
    polarityTotal = 0
    for value in polarityList:
        polarityTotal = polarityTotal + value
    
    polarityAVG = polarityTotal/len(polarityList)
    #print(polarityList)
    return polarityList, polarityAVG

In [422]:
# This function takes about a minute to run

# store all averages, then store chapter avg, also overall average
silm_polarity_avg = []
hobbit_polarity_avg = []
fellowship_polarity_avg = []
twotowers_polarity_avg = []
return_polarity_avg = []

silm_polarity_lists = []
hobbit_polarity_lists = []
fellowship_polarity_lists = []
twotowers_polarity_lists = []
return_polarity_lists = []
### Need to get chapter names in

for x in range(len(list_of_books)):
    book = list_of_books[x]
    segs = list_of_segments[x]
    
    if x == 0:
        print("silm")
    if x == 1:
        print("hobbit")
    if x == 2:
        print("fellowship")
    if x == 3:
        print("two towers")
    if x == 4:
        print("return of the king")
    
    for i in range(len(book)):
        if x == 0:
            temp1 = []
            temp2 = 0.0
            temp1,temp2 = Polarity(segs[i])
            silm_polarity_lists.append(temp1)
            silm_polarity_avg.append(temp2)
        if x == 1:
            temp1 = []
            temp2 = 0.0
            temp1,temp2 = Polarity(segs[i])
            hobbit_polarity_lists.append(temp1)
            hobbit_polarity_avg.append(temp2)
        if x == 2:
            temp1 = []
            temp2 = 0.0
            temp1,temp2 = Polarity(segs[i])
            fellowship_polarity_lists.append(temp1)
            fellowship_polarity_avg.append(temp2)
        if x == 3:
            temp1 = []
            temp2 = 0.0
            temp1,temp2 = Polarity(segs[i])
            twotowers_polarity_lists.append(temp1)
            twotowers_polarity_avg.append(temp2)
        if x == 4:
            temp1 = []
            temp2 = 0.0
            temp1,temp2 = Polarity(segs[i])
            return_polarity_lists.append(temp1)
            return_polarity_avg.append(temp2)
            
# chapter 3 of return of the king is super dark

silm
hobbit
fellowship
two towers
return of the king


In [406]:
print(silm_polarity_lists)
print(silm_polarity_avg)
for listy in silm_polarity_lists:
    print(len(listy))

[[4, 0, 0, 2, 4, 0], [0, 4, 0], [0, 0, 0, 0, 0, 2, 0, 2, 0, 4, 0], [4, 4, 0, 4, 0], [0, 4, 4, 0], [4, 0], [0, 0, 2, 0, 0, 0, 4, 4, 4, 0], [4, 4, 4, 4, 4, 4], [4, 0, 2, 0], [4, 0, 0, 0, 4, 4, 0], [0, 4, 2, 4, 2, 0, 0, 4], [4, 4, 4, 4, 4, 0], [0, 0, 0, 0, 4, 4, 0, 0, 2, 0, 0, 2, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0], [0, 0, 4, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 2, 0, 0, 0, 4, 0], [0, 0, 0, 4, 0, 0], [0, 0, 0, 4, 0, 0, 0, 0], [0, 4, 4, 4, 0, 0, 0, 0, 2, 0, 0, 4, 0, 0, 0, 4, 0, 2, 0, 0, 0], [0, 0, 0, 0, 0, 2, 4, 0, 0, 0, 0, 4, 0, 2, 0, 0, 2, 0, 0, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 2, 2, 2, 2], [4, 0, 4, 4, 4, 0, 4], [4, 4, 0], [4, 4, 4, 0, 0, 4, 0], [4], [0, 0, 4, 4], [4, 4, 0], [4, 4, 0, 0, 0]]
[1.6666666666666667, 1.3333333333333333, 0.7272727272727273, 2.4, 2.0, 2.0, 1.4, 4.0, 1.5, 1.7142857142857142, 2.0, 3.3333333333333335, 0.909090909090

In [430]:
## Pandas

silm_chapter_names = ["Ainundalë", "Valaquenta", "Of the Beginning of Days", "Of Aulë and Yavanna" , "Of the Coming of the Elves and the Captivity of Melkor",\
                     "Of Thingol and Melian", "Of Eldamar and the Princes of the Eldalië", "Of Fëanor and the Unchaining of Melkor", "Of the Silmarils and the Unrest of the Noldor",\
                     "Of the Darkening of Valinor", "Of the Flight of the Noldor", "Of the Sindar", "Of the Sun and Moon and the Hiding of Valinor", "Of Men", "Of the Return of the Noldor",\
                     "Of Beleriad and its Realms", "Of the Noldor in Beleriad", "Of Maeglin", "Of the Coming of Men into the West", "Of the Ruin of Beleriad and the Fall of Fingolfin", "Of Beren and Lúthien"\
                     "Of the Fifth Battle: Nirnaeth Arnoediad", "Of Túrin Turambar", "Of the Ruin of Doriath", "Of Tuor and the Fall of Gondolin", "Of the Voyage of Eärendil and the War of Wrath"]
hobbit_chapter_names = []
fellowship_chapter_names = []
twotowers_chapter_names = []
return_chapter_names = []

print(len(silm_chapter_names))
print(len(silm_chapters))



25
27


# Topic Modeling

# Testing

In [350]:
### Example JSON request
#NOTE: Sentiment140 Polarity values: 0: negative, 2: neutral, 4: positive

d = {'data':[{'text':'the titanic was ok'}, {'text':'this sucks'}]}
d['data'].append({'text':"Happy day!"})

r = requests.post('http://www.sentiment140.com/api/bulkClassifyJson?appid=blaz_j1@denison.edu', json=d)
js = r.json()

print(js['data'])

[{'text': 'the titanic was ok', 'polarity': 2, 'meta': {'language': 'en'}}, {'text': 'this sucks', 'polarity': 0, 'meta': {'language': 'en'}}, {'text': 'Happy day!', 'polarity': 4, 'meta': {'language': 'en'}}]


In [351]:
### Example of accessing the polarities

for i in range(len(d['data'])):
    print("Text:", js['data'][i]['text'], "\nPolarity:", js['data'][i]['polarity'])

Text: the titanic was ok 
Polarity: 2
Text: this sucks 
Polarity: 0
Text: Happy day! 
Polarity: 4


In [ ]:
## Segment list indexing examples

#print(hobbit_segments[15]) ## -- chapter
#print(hobbit_segments[0][0]) ## -- segments of chapter